## Project #2 - No Show Appointments

### 1. Data exploration in order to find dispensable columns and missing data

In [95]:
# Importing Modules
import pandas as pd
import matplotlib.pyplot as plt

# Importing data
df = pd.read_csv('data/noshowappointments-kagglev2-may-2016.csv')

# Ensuring that all columns will be lowercase
df.rename(columns=lambda x: x.lower(), inplace=True)

# Getting some data overview
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
patientid         110527 non-null float64
appointmentid     110527 non-null int64
gender            110527 non-null object
scheduledday      110527 non-null object
appointmentday    110527 non-null object
age               110527 non-null int64
neighbourhood     110527 non-null object
scholarship       110527 non-null int64
hipertension      110527 non-null int64
diabetes          110527 non-null int64
alcoholism        110527 non-null int64
handcap           110527 non-null int64
sms_received      110527 non-null int64
no-show           110527 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [96]:
df.head()

,patientid,appointmentid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


After *info()*, it is possible to know the number of samples (110527) and features (14) as well each feature data type

In [97]:
# Checking the number of unique values
df.nunique()

patientid          62299
appointmentid     110527
gender                 2
scheduledday      103549
appointmentday        27
age                  104
neighbourhood         81
scholarship            2
hipertension           2
diabetes               2
alcoholism             2
handcap                5
sms_received           2
no-show                2
dtype: int64

### 2. Data cleaning and transformation
As appointmentid feature is exclusive to each record (same number of entries and total samples), it can be droped. At a first glance, it seems that patientid could be removed once is a very specific value, but in a furder analysis it is possible to see that the number of unique values in patient id is less the total samples. It indicates that there are patients with more than one appointment. Hence, It is possible to identify those patients that have more no show appointments.

In [98]:
# Removing appointmentid
df.drop(['appointmentid'], axis=1, inplace=True)
df.nunique()

patientid          62299
gender                 2
scheduledday      103549
appointmentday        27
age                  104
neighbourhood         81
scholarship            2
hipertension           2
diabetes               2
alcoholism             2
handcap                5
sms_received           2
no-show                2
dtype: int64

If there are duplicates, they can be removed

In [99]:
# Checking duplicates
df.duplicated().sum()
# Droping duplicates
df.drop_duplicates(inplace=True)

Checking if there are null values

In [100]:
# Checking if there are null values
df.isnull().sum()

patientid         0
gender            0
scheduledday      0
appointmentday    0
age               0
neighbourhood     0
scholarship       0
hipertension      0
diabetes          0
alcoholism        0
handcap           0
sms_received      0
no-show           0
dtype: int64

It was decided to create two columns to keep the weekday of schedule and appointment respectively. It was considered important to check if some specifics weekdays have more no show occurrencies. 

Besides, It was decided to create other two columns to check if the schedule and appointment were made in the beginning (days between 1 and 10), middle (days between 11 and 20) or in the end of the month (days between 21 and 31). That was decided in order to check if there is any correlation between the period of the month and the amount of no shows.

In order to create those four columns, two functions were created. The first one *getMonthPeriodFromDate* [i] receives the date value as a string, once 'scheduledday' and 'appointmentday' has string values in original dataset, [ii] converts date value to DateTime and gets the day value. After that [iii] returns the period (beginning, middle or end).

The second function *getWeekDayFromDate()* receives the weekday as number end returns it as a string ('Monday' to 'Sunday')

Both functions were used to all items from 'scheduledday' and 'appointmentday' with 'map'. It was noticied that map function has a bad performance. Maybe there are alternatives to obtain the same result.

In [101]:
# Function that returns if a data is from beginning (1-10), middle (11-20) or end (20-31) of the month
def getMonthPeriodFromDate(dateAsStr):
    if pd.to_datetime(dateAsStr).day <= 10:
        return "beginning"
    elif pd.to_datetime(dateAsStr).day > 10 and pd.to_datetime(dateAsStr).day <= 20:
        return "middle"
    else:
        return "end"

In [102]:
# Function that returns the name of weekday
def getWeekDayFromDate(dateAsStr):
    switcher = {
        0: "Monday",
        1: "Tuesday",
        2: "Wednesday",
        3: "Thursday",
        4: "Friday",
        5: "Saturday",
        6: "Sunday"
    }
    return switcher.get(pd.to_datetime(dateAsStr).weekday())

In [103]:
# Function that returns the name of the month
def getMonthFromDate(dateAsStr):
    switcher = {
        1: "January",
        2: "February",
        3: "March",
        4: "April",
        5: "May",
        6: "June",
        7: "July",
        8: "August",
        9: "September",
        10: "October",
        11: "November",
        12: "December"
    }
    return switcher.get(pd.to_datetime(dateAsStr).month)

In [104]:
# Creating four new columns (two to get weekday and periodo of the month to scheduled day and two to appointment day)
# Each element of columns 'scheduledday' and 'appointmentday' is passed to the functions getMonthPeriodFromDate and getWeekDayFromDate by map 
# bad performance here :(

df['scheduled_weekday']         = df['scheduledday'].map(getWeekDayFromDate)
df['schedule_period_of_month'] = df['scheduledday'].map(getMonthPeriodFromDate)
df['schedule_month'] = df['scheduledday'].map(getMonthFromDate)

df['appointment_weekday']     = pd.to_datetime(df['appointmentday']).dt.weekday.map(getWeekDayFromDate)
df['appointment_period_of_month'] = df['appointmentday'].map(getMonthPeriodFromDate)
df['appointment_month'] = df['appointmentday'].map(getMonthFromDate)

df.head()

,patientid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no-show,scheduled_weekday,schedule_period_of_month,schedule_month,appointment_weekday,appointment_period_of_month,appointment_month
0,2.987250e+13,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No,Friday,end,April,Thursday,end,April
1,5.589978e+14,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No,Friday,end,April,Thursday,end,April
2,4.262962e+12,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No,Friday,end,April,Thursday,end,April
3,8.679512e+11,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,Friday,end,April,Thursday,end,April
4,8.841186e+12,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No,Friday,end,April,Thursday,end,April


In [105]:
df['days_between_sched_appoint'] = pd.to_datetime(df['appointmentday']) - pd.to_datetime(df['scheduledday'])
df.tail()

,patientid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no-show,scheduled_weekday,schedule_period_of_month,schedule_month,appointment_weekday,appointment_period_of_month,appointment_month,days_between_sched_appoint
110522,2.572134e+12,F,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56,MARIA ORTIZ,0,0,0,0,0,1,No,Tuesday,beginning,May,Thursday,beginning,June,34 days 14:44:25
110523,3.596266e+12,F,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51,MARIA ORTIZ,0,0,0,0,0,1,No,Tuesday,beginning,May,Thursday,beginning,June,34 days 16:32:27
110524,1.557663e+13,F,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21,MARIA ORTIZ,0,0,0,0,0,1,No,Wednesday,end,April,Thursday,beginning,June,40 days 07:56:08
110525,9.213493e+13,F,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38,MARIA ORTIZ,0,0,0,0,0,1,No,Wednesday,end,April,Thursday,beginning,June,40 days 08:50:37
110526,3.775115e+14,F,2016-04-27T13:30:56Z,2016-06-07T00:00:00Z,54,MARIA ORTIZ,0,0,0,0,0,1,No,Wednesday,end,April,Thursday,beginning,June,40 days 10:29:04


## 3. Exploring with visuals

In [111]:
#df.groupby(['gender', 'no-show']).size().unstack().plot(kind='bar',stacked=False);
df.gender.count()
#gb.get_group('F')
#df.groupby(['gender', 'no-show']).size().groupby(level=0).apply(lambda x: 100 * x / x.sum()).unstack().plot(kind='bar',stacked=True)
#plt.show()
#df['gender'].size
#df['no-show'].size

#df_f.groupby(['gender', 'no-show']).size().unstack().plot(kind='bar',stacked=True)
#df_m.groupby(['gender', 'no-show']).size().unstack().plot(kind='bar',stacked=True)


109909